In [72]:
import pandas as pd
import string
import nltk
nltk.download('stopwords')
from collections import Counter
from nltk.corpus import stopwords
import math
import numpy as np

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aadit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [73]:
stopwords_dict = Counter(stopwords.words('english'))

In [74]:
def docPreProcessing(filepath):
    file = open(filepath, 'r')
    lines = file.readlines()
    
    
    doctext = ""
    for line in lines:
        line = line.translate(str.maketrans('', '', string.punctuation)).strip().lower()
        line = ' '.join([word for word in line.split() if word not in stopwords_dict])
        doctext += line
        
    doctextList = doctext.split()
    return doctextList
        


In [75]:
def tf(wordlist):
    wordmap = {}
    
    for word in wordlist:
        if word in wordmap:
            wordmap[word] += 1
        else:
            wordmap[word] = 1
            
    
    numWords = len(wordmap)
#     print(numWords)
    
    for word in wordmap:
        wordmap[word] = wordmap[word] / numWords
    
    return wordmap
        


In [76]:
# tf(docPreProcessing('sample1.txt'))

In [77]:
def idf(doclist):
    
#     termSet = set()
    idfmap = {}
    
    for doc in doclist:
        wordmap = tf(docPreProcessing(doc))
        
        for term in wordmap:
            if term in idfmap:
                idfmap[term] += 1
            else:
                idfmap[term] = 1
                
    N = len(doclist)
                
            
    for term in idfmap:
        idfmap[term] = math.log(N/idfmap[term], 2)
        
    return idfmap
        

In [78]:
# doclist = []

# i = 1
# for i in range(1, 7):
#     doclist.append('sample' + str(i) + '.txt')
    
# print(idf(doclist))

In [83]:
def docMatrixBuilder(doclist, tf, idf):
    idfmap = idf(doclist)
    terms = list(idfmap)
    df = pd.DataFrame(0, index = terms, columns = doclist)
    
    for doc in doclist:
        wordmap = tf(docPreProcessing(doc))
        
        for word in wordmap:
            df.loc[word, doc] = wordmap[word]
        
        for term in idfmap:
            df.loc[term, doc] *= idfmap[term]
            
    
    
    return df

In [87]:
# def cosineSimilarityScore(q, D):
    

In [96]:
doclist = []

i = 1
for i in range(1, 7):
     doclist.append('sample' + str(i) + '.txt')
        
tfidfDataframe = docMatrixBuilder(doclist, tf, idf)
tfidfMatrix = tfidfDataframe.to_numpy().round(decimals=4)

# print(tfidfDataframe)
print(tfidfMatrix[:, 0].shape)

# # docvectors = []

# for d1 in doclist:
#     for d2 in doclist:
#         d1_v = tfidfMatrix.loc[:, d1].to_numpy
#         d2_v = tfidfMatrix.loc[:, d2].to_numpy
#         angle = cosineSimilarityScore(d1_v, d2_v)
        
#         print(d1 + " vs " + d1 + ": " + angle)
        
        
        


# cosineSimilarityScore(tdidfMatrix)


(679,)
